# Explanatory section

## Objective
- this section is for explaining the implementation of the Walklets algorithm  
and the other methods that were relevant for the testing phase  

- this section also describes what problems were encountered and how were solved  
in the development stage of this paper


**Paper used for implementation:**  
**"Don't Walk, Skip! Online Learning of Multi-scale Network Embeddings"**-->
<https://arxiv.org/abs/1605.02115> 

## Algorithm explanations

- in this section I will describe the methods used from the `WalkletsEmbedder` class
and also the methods that were used at the testing phase of the algorithm

- first the project has the following essential files:
- the *walklets_test.ipynb*, used for testing the algorithm implemented,  
presents the implementation step-by-step in the **Implementation of algorithm** section
- the *utils* is a module(package) used for functions called at the test phase
- it contains the *utils.py* and since gensim was not upgraded to use the last numpy vers.  
I used the *word2vec.py* reimplementation for the Walk2Vec Embedding Model
- the files *random_walks.py* and *walklets.py* contains the implementation of both algorithms,  
RandomWalk and Walklets which share the same platform(structure) for implementation

### utils.py

- as mentioned above i will present some methods that I've added for visualising the results from  
test phase

- utils module contains utils.py, which was modified, and the word2vec.py and pyg_utils.py
- this module was also used during the laboratories, and it is credited to
https://github.com/zademn

#### `plot_similarity_heatmap()`

In [ ]:
def plot_similarity_heatmap(
    embeddings: np.ndarray,
    node: int = 0,
    cmap: str = "hot_r",
    graph: nx.Graph = None,
    labels: np.ndarray = None,
    opt: int = 0,
    figsize: tuple = (7, 7),
    dpi: int = 100,
    node_size: int = 20
):
    """
    Plots t-SNE projection with class-based coloring (opt=0) or graph-colored by similarity to node (opt=1).

    Args:
        embeddings (np.ndarray): Embedding matrix.
        node (int): Root node to compute similarity from (only used in opt=1).
        cmap (str): Colormap.
        graph (nx.Graph, optional): The graph to draw (required for opt=1).
        labels (np.ndarray, optional): Node labels (required for opt=0).
        opt (int): 0 = t-SNE with labels, 1 = graph colored by softmax similarity to root.
        figsize (tuple): Figure size.
        dpi (int): Plot resolution.
        node_size (int): Node size for graph plot.
    """
    if embeddings is None:
        raise ValueError("Embeddings are not initialized. Please fit the model first.")

    if opt == 0:
        if labels is None:
            raise ValueError("Labels must be provided when opt=0 (t-SNE visualization).")

        tsne_emb = TSNE(n_components=2, random_state=42).fit_transform(embeddings)
        plt.figure(figsize=figsize)
        plt.scatter(tsne_emb[:, 0], tsne_emb[:, 1], c=labels, cmap=cmap)
        plt.title("t-SNE projection of embeddings (colored by labels)")
        plt.axis("off")
        plt.show()
    else:
        if graph is None:
            raise ValueError("Graph must be provided when opt=1 (graph visualization).")

        raw_sims = embeddings @ embeddings[node].T
        norm_sims = (raw_sims - raw_sims.min()) / (raw_sims.max() - raw_sims.min())*250
        norm_sims[node]=0

        pos = nx.spring_layout(graph, seed=42)
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
        nx.draw_networkx(
            graph,
            pos=pos,
            with_labels=False,
            node_color=norm_sims,
            cmap=cmap,
            node_size=node_size,
            ax=ax
        )
        plt.title(f"Graph coloring based on similarity rooted from node {node}")
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_array(norm_sims)
        fig.colorbar(sm, ax=ax, label="Normalized Similarity*250")
        plt.show()

#####  Explanation

This function visualizes graph embeddings in two ways:

**Option 0 — t-SNE Projection:**
- Projects high-dimensional embeddings to 2D using **t-SNE**.
- Colors each node based on its **class label**.
- Ideal for visualizing how well the embedding separates different classes.
- Used when performing community metric evaluation of larger networks

**Option 1 — Graph Colored by Similarity:**
- Computes **scaled dot-product similarity** between a selected `node`=root_node and all others using their embeddings.
- Visualizes the graph using **NetworkX**, coloring nodes by their similarity.
- Useful for analyzing how a node's neighborhood behaves in embedding space.
- Used when performing community metric evaluation of smaller networks

#####  What was modified?

- Initially there was a single option, that to plot only the graph by networkx draw, i wanted to use the  
`draw_kamada_kawai` but this methodhad far more way parameters thus making it more tunable

- When i choosed the datasets I've decided to use only a single network, a small one, but beacuse of a train time  
pretty short, i've decided to intergate another dataset, medium to large one, that of Cora
- When i've decided to test also on the large dataswet, i've created the option 0, that intially,  
printed the heatmaps of every node with a color by its similarity(scale-dot product), but it wasnt feasable as i cannot see,  
in an entire screen all the nodes heatmaps, or they were very small.
- Then i've decied to move on plotting mechanism, and by this, isntaed of using the draw_networkx , that could be  
very expensive in graphic generation, cause we have an enomrous quantity of nodes to be plotted, i've used plt from  
`matplotlib` that plots the embedds, of course reduced from their intiial 128 dim to 2d dim by TSNE manifold learning algorithm
- For option 1 or whatever is different than 0 initially i've used softmax applied to the dot products to see the simillarities,  
as probabilities, but by testing i saw that it doesn't represent the dot-product very well, so i've switched to using the scale-dot  
as normalized similarities, then multipling to 250 as the cmap has values ranging [0,250]
- The cmap used is `hot_r` giving high to low similarities dark to lighter colors

#### `split_graph()`

In [ ]:
def split_graph(
    original_graph: nx.Graph,
    labels: dict,
    train_percentage: float,
    seed: int = 42
):
    # 1. Set seed for reproducibility
    random.seed(seed)

    # 2. List of all nodes
    all_nodes = list(original_graph.nodes())

    # 3. Shuffle and split
    random.shuffle(all_nodes)
    train_size = int(train_percentage * len(all_nodes))
    train_nodes = set(all_nodes[:train_size])
    test_nodes = set(all_nodes[train_size:])

    # 4. Build two subgraphs
    graph_train = original_graph.subgraph(train_nodes).copy()
    graph_test = original_graph.subgraph(test_nodes).copy()

    # 5. Split labels
    labels_train = {node: labels[node] for node in train_nodes if node in labels}
    labels_test = {node: labels[node] for node in test_nodes if node in labels}

    return graph_train, labels_train, graph_test, labels_test

##### Explanation

This function is used to **split a graph and its node labels into a training and testing subset**, preserving the graph structure within each split.

To train the classification model on one part of the graph and evaluate it on another, we need to:
- Randomly divide the nodes into training and test sets.
- Create two subgraphs based on the original_graph: one for training and one for testing.
- Assign the labels accordingly, to each node choosed randomly  

The nodes returned will be used for searching their embeddings through `get_embedding_specific` or `get_scale_embedding_specific`  
and feeding them to the classif model

As we have the `train_test_split` method for dividing the features, the split_graph its build on the same concept, but  
dividing nodes of the graph(network) instead of features of a dataset


### walklets.py

- this is the file in which the Walklets algorithm is implemented

- in the first part of the file, there is short explanation of what are Walklets, what does the emb preserves compared  
to original RandomWalk corpus 1-scaled(k=1)
- the method shares the same platform as the *random_walks.py* file, as it has the same methods,  
but it also exists cases where we use the methods from rw, like walk_graph
- the methods suffered many changes along the development process
- the algorithm its presented with its fields, and methods in the `WalkletsEmbedder` class

#### `__init__()`

In [ ]:
def __init__(
        self,
        seed: int = 42,
        walk_number: int = 1000,
        walk_length: int = 11,
        embed_dim: int = 128,
        window_size: int = 3,
        workers: int = 4,
        epochs: int = 250,#enough for training time
        learning_rate: float = 0.025,
    ):
        """
        The method initializes the Random Walk algorithm.
        Returns:
            nothing
        Args:
            seed (int): Random seed value. Default is 42.
            walk_number (int): Number of random walks. Default is 1000.
            walk_length (int): Length of random walks. Default is 11.
            embed_dim (int): Dimensionality of embedding. Default is 128.
            window_size (int): Size of the context window. Default is 3
            e.g. walk->[0,1,2,3,4,5] and window_size=3 for node 4 word2vec will use as target: (4,1),(4,2),(4,3),(4,5)
            workers (int): Number of cores to be used by the Word2Vec Model. Default is 4.
            epochs (int): Number of epochs. Default is 500.
            learning_rate (float): Learning rate of the Word2Vec Model. Default is 0.025.
        """
        self.walk_number = walk_number
        self.walk_length = walk_length
        self.embed_dim = embed_dim
        self.window_size = window_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.workers = workers
        self.seed = seed
        self.embeddings = None
        self.node_to_idx = None
        np.random.seed(self.seed)  # Set the random seed

##### Explanation

- as simple as it is, this file its the constructor of the `WalkletsEmbedder` class, and it instances  
the params of the model with the actual parameters that has got or if they are not given, the constructor  
uses a standard parameters

- along this parameters we have  
walk_number: int = 1000, value used in the ResearchPaper  
walk_length: int = 11, value used in the ResearchPaper  
embed_dim: int = 128, value used in the ResearchPaper  
epochs: int = 250, enough for training time, due to hardware limitations  
learning_rate: float = 0.025, value used in the ResearchPaper
- the workers arg offers a multitasking capability that wasn't implemented anymore
- we also use for this class a dict where we save the embeddings for each scale-->`self.scale_emebeddings`
- this feild will be useful when storing the trained emb. and also when we will want to search for a emb of a node
- beacause we store the emb for each scales idnivdually we are not concatenate them, so we will not use the `self.embeddings` field
- we also use in this class the scales, an array which will store the int 's of the scales that were used and present in the  
`self.scale_embeddings`
- this will be useful when a user wants to get the embeddings for a scale or the embedding for a node contained in a scale, so by this  
we verify if the user wants a scale that was fitted first
- and we also use a dict type `self.node_to_idx` to store {str of the node:index of embed} so we can access later on  
the embeddings by having a list of nodes List[Any] given as parameters for another methods that will find their emb.

##### What was modified?

- at the beggining versions the algorithm, could make embeddings just for a single scale, and store them all in the  
`self.embeddings`

- but when decided to train also on a larger networks, this structure proven to be very high time and resource consumer,  
because at each scale we would train the algorithm, would need to remake the randomwalk(the initial corpus)
- so i compressed all scales in a single fit call that gets a list of scales, and the randomwalk are made once, and then on it  
is made the corpus for the walklets and init of word2vec and training process, and the traiend embedds are stored in an organized  strcuture `self.scale_embeddings`

#### `select_walklets()`

In [ ]:
 @staticmethod
    def _select_walklets(walks: List[List[str]], scale: int=2) -> List[List[str]]:
        """
        Extracts nodes that are separated by 'scale' steps in each walk to form a new corpus.
        Args:
            walks (List[List[str]]): The list of walks where each walk is a list of node IDs as strings
            scale (int): The scale k at which to extract nodes   
        Returns:
            List[List[str]]: A list of walklet sequences with nodes that are scale steps apart
        """
        walklets = []
        for walk in walks:
            if len(walk) <= scale:
                continue
            
            # Create a new walk with nodes that are scale steps apart
            walklet = [walk[i] for i in range(0, len(walk), scale)]
            if len(walklet) > 1:  # Only add walks with at least 2 nodes
                walklets.append(walklet)
        
        return walklets

##### Explanation

This static method takes a list of random walks and selects only those nodes that are separated by a **fixed number of steps (`scale`)** within each walk.

It's a core part of the **Walklets algorithm**, which captures multi-scale context by:
- Skipping over intermediate nodes.
- Producing a new "corpus" of node sequences where only distant relationships are preserved.

This means that a node will capture information about nodes that are at a farther distance, e.g another comunity
- Walklets use this method to create different views of the graph at **multiple scales** (e.g., local vs. global structure).
- These walklets are then fed into Word2Vec to learn embeddings that capture relationships at scale `k`.

- First it checks if the scale needed its far way larger than the `walk_length`, implcitly the length of the current walk, cause its the max limit a scale could be
- Then if lower it will iterate with a for on the current walk, with step=scale, obtaining a new list with indexes extracted at scale steps 

*Observe that the method its a static one, we could call it once we have the walks generated by walk_graph() from RandomWalkEmbedder*

#### `fit()`

In [ ]:
    def fit(self, original_graph: nx.classes.graph.Graph = nx.karate_club_graph(), scales: Union[List[int], int] = 1):
        """
        The method fits the Walklets model to the graph.
        It will create random walks on the graph provided, then for each scale it will
        extract nodes that are k distance apart from these walks, creating separate corpus for each scale.
        Then it will train a Word2Vec model for each scale and concatenate all embeddings.
        
        Returns:
            nothing
        Args:
            original_graph (nx.Graph): The original graph to be used for embedding. Default is the karate club graph.
            scales (Union[List[int], int]): The scale(s) of the walklets. Can be a single integer or a list of integers.
                                           Default is 1 (equivalent to RandomWalk).
        """
        # Convert single scale to list for consistent processing
        if isinstance(scales, int):
            scales = [scales]
        
        # Save scales used
        self.scales = scales
        
        # Initialize node to index mapping
        self.node_to_idx = {str(node): idx for idx, node in enumerate(original_graph.nodes())}
        num_of_nodes = original_graph.number_of_nodes()
        
        # Generate random walks using the static method from RandomWalkEmbedder
        # We generate the walks once and reuse them for different scales
        walks = RandomWalkEmbedder.walk_graph(original_graph, self.walk_number, self.walk_length)
        print(f"Generated a total of {len(walks)} random walks of length {self.walk_length}, starting from {len(walks)/self.walk_number} nodes.\n")
        
        # Initialize embeddings array
        all_embeddings = []
        
        # Process each scale
        for scale in scales:
            print(f"Processing scale k={scale}")
            
            # Extract node sequences that are 'scale' steps apart
            scale_corpus = self._select_walklets(walks, scale)
            print(f"Extracted {len(scale_corpus)} walklets for scale k={scale}")
            
            # Train custom Word2Vec model
            model = Word2Vec(
                sequences=scale_corpus,
                embedding_dim=self.embed_dim,
                window_size=self.window_size,  # Using standard window size for the new corpus
                negative_samples=5  # Default value for negative samples
            )
            
            # Train the model with specified epochs and batch size
            model.train(epochs=self.epochs, batch_size=1024)
            print("\n")
            
            # Create the embeddings array for this scale
            scale_embeddings = np.zeros((num_of_nodes, self.embed_dim))
            for node in original_graph.nodes():
                node_str = str(node)
                vector = model.get_word_vector(node_str)
                if vector is not None:
                    scale_embeddings[self.node_to_idx[node_str]] = vector
            
            # Store this scale's embeddings
            self.scale_embeddings[scale] = scale_embeddings

##### Explanation

This method trains the **Walklets** model on a given graph. It performs the following steps:

1. **Prepare scales**: Convert a single scale value to a list for unified processing.

2. **Generate random walks**: Reuse these walks across all scales.
3. **For each scale `k`**:
   - Select walklets by skipping `k-1` nodes in walks.(make the corpus via `select_walks`)
   - Train a Word2Vec model on these walklets.
   - Extract node embeddings and store them by scale.

- This method uses batch_size=1024 for organizing the dataset used for Word2Vec

- After the Word2Vec trains the embedds, we take all the nodes from the graph given as the parameter, and  
get through `get_word_vector` the emb of that node, than by using the `self.node_to_idx` we store the emb, in the  
`self.scale_embeddings`

##### What was modified?

- As stated above, the agorithm here was modified, intially from computing the rwalks then computing the walklets for a single scale, then training the Word2Vec, to computing the rwalks once and compute the walklets and train the emb for multiple scales, scales given  through `scales` paramater as a List of int' s or an int (stored as union structure type, mutual exclusive structure)   

#### `get_scale_embedding()` && `get_scale_embedding_specific()`

In [ ]:
def get_scale_embedding(self, scale: int) -> np.ndarray:
        """
        The method returns embeddings for a specific scale.
        
        Args:
            scale (int): The scale for which to get embeddings
            
        Returns:
            np.ndarray: A 2D array of node embeddings for the specified scale
            
        Raises:
            ValueError: If the model has not been fitted or if the scale is not available
        """
        if not self.scale_embeddings:
            raise ValueError("Model has not been fitted. Call fit() first.")
        
        if scale not in self.scales:
            raise ValueError(f"Scale {scale} not found. Available scales: {self.scales}")
        
        return self.scale_embeddings[scale]
    
    def get_scale_embedding_specific(self, nodes: List[Any], scale: int) -> np.ndarray:
        """
        The method gets embeddings only for the specified list of nodes at a specific scale.
    
        Args:
            nodes (List[Any]): List of node IDs (the IDs as they are in the graph)
            scale (int): The scale to get embeddings for
        
        Returns:
            np.ndarray: Embeddings for the specified nodes at the specified scale.
                        Shape (len(nodes), embed_dim)
                   
        Raises:
            ValueError: If the model has not been fitted or if the scale is not available
        """
        if not self.scale_embeddings or not self.scales:
            raise ValueError("Model has not been fitted. Call fit() first.")
    
        if scale not in self.scales:
            raise ValueError(f"Scale {scale} not found. Available scales: {self.scales}")
    
        # Get the embeddings for the specified scale
        scale_embeddings = self.scale_embeddings[scale]
    
        # Get the indices for the specified nodes
        idxs = [self.node_to_idx[str(node)] for node in nodes]
    
        # Return the embeddings for the specified nodes at the specified scale
        return scale_embeddings[idxs]
    

##### Explanation

After training the Walklets model, we often want to:

- Extract the **entire embedding matrix** for a given scale `k`.

- Retrieve **specific node embeddings** for a subset of nodes at scale `k`.

Such operations are descirbed byt the following methods:

**Method 1**: `get_scale_embedding(scale)`
- Returns the full embedding matrix for a specified scale(all the node's embeddings for a scale)
- Shape returned: `(len(self.scale_embeddings[...]), embedding_dim)`

**Method 2**: `get_scale_embedding_specific(nodes, scale)`
- Returns embeddings only for a given list of nodes at a specified scale.
- Shape: `(len(nodes), embedding_dim)`

- This methods are based on the `get_embedding` and `get_embedding_specific` from `RandomWalkEmbedder` class  
but working on different scales rather than a sngle scale k=1, we will build a method that gets the embeddings,  
for all the nodes trained on a scale=k, and for the other one we will build a method that gets the emb of the nodes  
gaved by param from a given scale

#### `store_emb()` && `load_emb()`

In [ ]:
 def store_emb(self, path: str):
        """
        Stores the embeddings and node mapping to files.
        Args:
            path (str): Path where to save the embeddings
        """
        if not self.scale_embeddings or not self.scales:
            raise ValueError("Model has not been fitted. Call fit() first.")
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        # Save individual scale embeddings
        for scale in self.scales:
            scale_path = os.path.join(os.path.dirname(path), os.path.basename(path) + f"_scale_{scale}_embeddings.npy")
            np.save(scale_path, self.scale_embeddings[scale])
        
        # Save scales list
        scales_path = os.path.join(os.path.dirname(path), os.path.basename(path) + "_scales.json")
        with open(scales_path, 'w') as f:
            json.dump(self.scales, f)
        
        # Save node mapping
        mapping_path = os.path.join(os.path.dirname(path), os.path.basename(path) + "_mapping.json")
        with open(mapping_path, 'w') as f:
            json.dump({str(k): int(v) for k, v in self.node_to_idx.items()}, f)
    
    def load_emb(self, path: str) -> Tuple[Dict[str, int], List[int], Dict[int, np.ndarray]]:
        """
        Loads embeddings, node mapping, and scales from files.
        Args:
            path (str): Path to the saved embeddings (without file extensions)
        Returns:
            Tuple[Dict[str, int], List[int], Dict[int, np.ndarray]]: 
                - Node to index mapping
                - List of scales
                - Dictionary of scale-specific embeddings
        """
        # Construct paths
        mapping_path = os.path.join(os.path.dirname(path), os.path.basename(path) + "_mapping.json")
        scales_path = os.path.join(os.path.dirname(path), os.path.basename(path) + "_scales.json")

        if not os.path.exists(mapping_path) or not os.path.exists(scales_path):
            raise FileNotFoundError("Mapping or scales file not found.")
    
        # Load node mapping
        with open(mapping_path, 'r') as f:
            self.node_to_idx = json.load(f)
            self.node_to_idx = {k: int(v) for k, v in self.node_to_idx.items()}
    
        # Load scales
        with open(scales_path, 'r') as f:
            self.scales = json.load(f)
    
        # Load scale-specific embeddings
        for scale in self.scales:
            scale_path = os.path.join(os.path.dirname(path), os.path.basename(path) + f"_scale_{scale}_embeddings.npy")
            if os.path.exists(scale_path):
                self.scale_embeddings[scale] = np.load(scale_path)
            else:
                raise FileNotFoundError(f"Embeddings for scale {scale} not found")

##### Explanation

This methods are useful for large networks, for storing their trained params.

- Because we will have large datasets(Cora) to train on, we will have a bigger execution time
- Instead of training once more, we will store the emb once the `fit` was called, then whenever  
when you want to test the emb, you will, instead of calling `fit`, call `load` and use the test cases in the  
`walklets_test.ipynb` as normal
- Observe that store, will store not only the embeddings but the other relevant parameter too
- The model will save its `self.node_to_idx`, so when we will load, we could use the methods
`get_scale_embedding_specific` and `get_scale_embedding` without problems
- In this way the model will save its `self.scale_embeddings` and `self.scales` to be used after we call the `load`

##### What was modified?

- Intially when i've tested on a small network(KarateClub) only, it wouldn't be ncecesarly to store the model's parameters

- But as we use larger and larger networks , the time needed for executing the training phase will grow expon.
- As we will want to test the emb through different tests later on, **this step is mandatory**

### random_walks.py

-  this file implements the RandomWalk algorithm using the `RandomWalkEmbedder` class

- the RandomWalk algorithm uses in the same fashion as Walklets the corpus obtained via RandomWalk algorithm,  
but this time leaving the walks as they are, at k=scale=1

- the methods are likewise `WalkletsEmbedder` class, but instead of using different scales we will use a single scale, k=1
- this means that all the methods that used mutliple scales will change to use a single scale
- by this we reffer to the `fit` method, which will only make the rwalks through `walk_graph` method  
and train directly on that corpus
- by this we reffer to the `get_embedding` and `get_embedding_specific` methods, which will return all the embeddings  
here stored in the field `self.embeddings`, respectively return the emb for certain nodes
- in the store and load we store and load only the `self.embeddings` and `self.node_to_idx`
- all the other methods ar the same as `WalkletsEmbedder` class
- the `walk_graph` and `walk_node` will be static methods , which, will  

1. `walk_graph`-->for all the nodes in the graph will call `walk_node` for `walk_number` times ,making `walk_number` walks, that  
are preserved in a List[]
2. `walk_number`-->makes a walk by randomly choosing on which neigbour of the current node to move foward, than making this for `walk_length` times, thus creating a walk that will be stored in the List of walks  